# PROJET APPLICATION DE RECOMMANDATION DE FILMS

## PARTIE 1 : KPI

In [2]:
import os
import io
import gzip
import random
import secrets
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
import plotly.io as pio
from sklearn import tree
import plotly.express as px
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from flask import Flask, request, render_template, session, url_for, redirect

#### chargement des datasets de type TSV

In [ ]:
# scrapping des noms des fichiers
url = "https://datasets.imdbws.com/"

# contournement d'éventuels protection anti-scrapping
navigator = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

response = requests.get(url, headers={"User-Agent": navigator})
# response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Trouver tous les liens sur la page
liens = soup.find_all("a")

# création de la liste des fichiers
fichiers = []
for lien in liens:
    #  choisir uniquement les liens de type href
    nom_fichier = lien.get("href")
    #  choisir uniquement les liens se terminant par .tsv.gz
    if nom_fichier.endswith(".tsv.gz"):
        # print(nom_fichier)
        fichiers.append(nom_fichier)
print(f"\n fichiers:\n{fichiers} \n")

# Liste des noms de fichiers à télécharger
# fichiers = ['title.principals.tsv.gz',
#             'title.akas.tsv.gz',
#             'name.basics.tsv.gz',
#             'title.basics.tsv.gz',
#             'title.crew.tsv.gz',
#             'title.episode.tsv.gz',
#             'title.ratings.tsv.gz' ]

In [ ]:
# Dictionnaire pour stocker les dataframes
dfs = {}

for fichier in fichiers:
    # url = f'https://datasets.imdbws.com/{fichier}'
    response = requests.get(fichier)
    gzip_file = gzip.open(io.BytesIO(response.content), "rt", encoding="utf-8")
    df = pd.read_csv(gzip_file, sep="\t", low_memory=False)
    # Chaque dataframe porte le nom du fichier sans l'extension
    nom_sans_extension = os.path.splitext(os.path.basename(fichier))[0]
    # suppression du tz et remplacement des . par des _
    nom_sans_extension = os.path.splitext(nom_sans_extension)[0]
    nom_sans_extension = nom_sans_extension.replace(".", "_")
    print(f"\nnom_sans_extension :\n{nom_sans_extension} \n")
    dfs[nom_sans_extension] = df

    # dfs[fichier] = df

In [11]:
# liste des datafdrames
for df in dfs:
    print(f"{df} ")

name_basics 
title_akas 
title_basics 
title_crew 
title_episode 
title_principals 
title_ratings 


In [44]:
# Résumé des informations des dataframes
for df in dfs:
    print(f"\ndf :{df} ")
    your_dataframe = dfs[df]
    print(f"shape: {your_dataframe.shape} - list columns :{your_dataframe.columns.tolist()} \nliste des colonnes numeriques: {your_dataframe.select_dtypes(include=[np.number]).columns.tolist()} \nliste des colonnes non numeriques: {your_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()} \nColonne avec des na :\n{your_dataframe.isna().sum()}")
    print(f"Noms des colonnes avec au moins une valeur NA : {your_dataframe.columns[your_dataframe.isna().any()].tolist()}")
    print(f"Nombre de lignes avec au moins une valeur NA : {your_dataframe.isna().any(axis=1).sum()}")
    print(f"head : \n{your_dataframe.head(2)}")


df :name_basics 
shape: (13144850, 6) - list columns :['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'] 
liste des colonnes numeriques: [] 
liste des colonnes non numeriques: ['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'] 
Colonne avec des na :
nconst                     0
primaryName                7
birthYear                  0
deathYear                  0
primaryProfession    2630039
knownForTitles             0
dtype: int64
Noms des colonnes avec au moins une valeur NA : ['primaryName', 'primaryProfession']
Nombre de lignes avec au moins une valeur NA : 2630045
head :       nconst    primaryName birthYear deathYear  \
0  nm0000001   Fred Astaire      1899      1987   
1  nm0000002  Lauren Bacall      1924      2014   

                primaryProfession                           knownForTitles  
0  soundtrack,actor,miscellaneous  tt0072308,tt0053137,tt0050419,tt0031983  
1              actress,soun

In [50]:
# création d'un fichier de synthése des dataframes au format markdown
import datetime
# date et heure actuelles
now = datetime.datetime.now()
# Formatage date et heure au format 2024-01-01-9h30
date_time = now.strftime("%Y-%m-%d-%Hh%M")
# Ajouter la date et l'heure au nom du fichier
filename = f"synthése_dataframes_{date_time}.md"
with open(filename, "w", encoding="utf-8") as f:
    for df in dfs:
        your_dataframe = dfs22222[df]
        f.write(f"- **head** : \n```{your_dataframe.head(2)}```\n")

In [53]:
# création d'un fichier de synthése des dataframes au format markdown
import datetime
# date et heure actuelles
now = datetime.datetime.now()
# Formatage date et heure au format 2024-01-01-9h30
date_time = now.strftime("%Y-%m-%d-%Hh%M")
# Ajouter la date et l'heure au nom du fichier
filename = f"synthése_dataframes_{date_time}.md"
with open(filename, "w", encoding="utf-8") as f:
    for df in dfs:
        your_dataframe = dfs[df]
        f.write(f"\n## DataFrame : {df}\n")
        f.write(f"- **Shape** : `{your_dataframe.shape}`\n")
        f.write(f"- **Liste des colonnes** : `{your_dataframe.columns.tolist()}`\n")
        f.write(f"- **Liste des colonnes numériques** : `{your_dataframe.select_dtypes(include=[np.number]).columns.tolist()}`\n")
        f.write(f"- **Liste des colonnes non numériques** : `{your_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()}`\n")
        f.write(f"- **Colonnes avec des NA** :\n```\n{your_dataframe.isna().sum()}\n```\n")
        f.write(f"- **Noms des colonnes avec au moins une valeur NA** : `{your_dataframe.columns[your_dataframe.isna().any()].tolist()}`\n")
        # f.write(f"- **Nombre de lignes avec au moins une valeur NA** : `{your_dataframe.isna().any(axis=1).sum()}`\n")
        # f.write(f"- **head** : \n```\n{your_dataframe.head(2)}\n```\n")
        # Obtenir les en-têtes de colonnes
        headers = ' | '.join(your_dataframe.columns.tolist())

        # Obtenir les lignes
        rows = [' | '.join(row) for row in your_dataframe.head(2).astype(str).values]

        # Écrire les en-têtes et les lignes dans le fichier
        f.write(f"- **head** :\n```\n{headers}\n{'\n'.join(rows)}\n```\n")

### Preprocessing dataframes

In [54]:
# création d'un fichier de synthése des dataframes au format markdown
import datetime
# date et heure actuelles
now = datetime.datetime.now()
# Formatage date et heure au format 2024-01-01-9h30
date_time = now.strftime("%Y-%m-%d-%Hh%M")
# Ajouter la date et l'heure au nom du fichier
filename = f"synthése_dataframes_{date_time}.md"
with open(filename, "w", encoding="utf-8") as f:
    for df in dfs:
        your_dataframe = dfs[df]
        f.write(f"\n## DataFrame : {df}\n")
        f.write(f"- **Shape** : `{your_dataframe.shape}`\n")
        f.write(f"- **Liste des colonnes** : `{your_dataframe.columns.tolist()}`\n")
        f.write(f"- **Liste des colonnes numériques** : `{your_dataframe.select_dtypes(include=[np.number]).columns.tolist()}`\n")
        f.write(f"- **Liste des colonnes non numériques** : `{your_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()}`\n")
        f.write(f"- **Colonnes avec des NA** :\n```\n{your_dataframe.isna().sum()}\n```\n")
        f.write(f"- **Noms des colonnes avec au moins une valeur NA** : `{your_dataframe.columns[your_dataframe.isna().any()].tolist()}`\n")
        f.write(f"- **Nombre de lignes avec au moins une valeur NA** : `{your_dataframe.isna().any(axis=1).sum()}`\n")
        headers = ' | '.join(your_dataframe.columns.tolist())
        # Obtenir les lignes
        rows = [' | '.join(row) for row in your_dataframe.head(2).astype(str).values]
        # Écrire les en-têtes et les lignes dans le fichier
        f.write(f"- **head** :\n```\n{headers}\n{'\n'.join(rows)}\n```\n")

In [68]:
df=dfs["name_basics"].copy()
df.explode("knownForTitles")


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0056404,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0078723,tt0080455,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0050976,tt0069467,tt0083922"
...,...,...,...,...,...,...
13144845,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
13144846,nm9993716,Essias Loberg,\N,\N,NaN,\N
13144847,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
13144848,nm9993718,Aayush Nair,\N,\N,cinematographer,tt8736744


Voici quelques suggestions pour le prétraitement de vos DataFrames :

1. **name_basics** : Vous pouvez convertir les colonnes `birthYear` et `deathYear` en type numérique. De plus, la colonne `primaryProfession` semble contenir plusieurs professions pour une même personne, vous pourriez diviser cette colonne en plusieurs colonnes binaires (une pour chaque profession).

2. **title_akas** : La colonne `isOriginalTitle` pourrait être convertie en booléen. De plus, si vous n'avez pas besoin de toutes les régions ou langues, vous pourriez filtrer ces colonnes pour ne garder que les lignes pertinentes.

3. **title_basics** : Les colonnes `startYear` et `endYear` pourraient être converties en type numérique. La colonne `genres` semble contenir plusieurs genres pour un même titre, vous pourriez diviser cette colonne en plusieurs colonnes binaires (une pour chaque genre).

4. **title_crew** : Les colonnes `directors` et `writers` semblent contenir plusieurs personnes pour un même titre, vous pourriez diviser ces colonnes en plusieurs colonnes (une pour chaque personne).

5. **title_episode** : Les colonnes `seasonNumber` et `episodeNumber` pourraient être converties en type numérique.

6. **title_principals** : La colonne `characters` semble contenir plusieurs personnages pour une même personne, vous pourriez diviser cette colonne en plusieurs colonnes (une pour chaque personnage).

7. **title_ratings** : Les colonnes `averageRating` et `numVotes` pourraient être converties en type numérique.

En général, il est recommandé de gérer les valeurs manquantes, de supprimer les doublons, et d'effectuer une normalisation ou une standardisation des données si nécessaire. De plus, selon votre objectif, vous pourriez avoir besoin de fusionner ces DataFrames en utilisant les colonnes appropriées comme clés. 

N'oubliez pas que ces suggestions dépendent de votre objectif final et du modèle que vous prévoyez d'utiliser. Assurez-vous de comprendre les implications de chaque étape de prétraitement avant de l'appliquer à vos données. Bonne chance avec votre analyse ! 😊

Pour calculer les KPI de vos DataFrames, vous pouvez envisager les mesures suivantes :

1. **name_basics** : Nombre total d'acteurs, nombre d'acteurs par profession, nombre d'acteurs nés chaque année, etc.
2. **title_akas** : Nombre total de titres, nombre de titres par région ou par langue, etc.
3. **title_basics** : Nombre total de titres par type (film, série, etc.), durée moyenne des titres, nombre de titres par genre, etc.
4. **title_crew** : Nombre de réalisateurs et scénaristes uniques, nombre moyen de réalisateurs et scénaristes par titre, etc.
5. **title_episode** : Nombre total d'épisodes, nombre moyen d'épisodes par saison, etc.
6. **title_principals** : Nombre moyen de personnages par titre, nombre de titres par acteur, etc.
7. **title_ratings** : Note moyenne des titres, nombre moyen de votes par titre, etc.

Pour proposer 10 suggestions de films en fonction du choix d'un film d'un utilisateur, vous pouvez utiliser une approche de filtrage collaboratif ou basée sur le contenu. Voici une approche simple basée sur le contenu :

1. **Sélectionnez le film choisi par l'utilisateur** dans votre DataFrame `title_basics`.
2. **Identifiez les genres** de ce film.
3. **Filtrez** votre DataFrame `title_basics` pour ne garder que les films qui partagent au moins un genre avec le film choisi par l'utilisateur.
4. **Classez** ces films en fonction de leur note moyenne dans votre DataFrame `title_ratings` (vous pouvez également prendre en compte le nombre de votes pour éviter les films avec une note moyenne élevée mais un faible nombre de votes).
5. **Sélectionnez les 10 premiers films** de cette liste.

N'oubliez pas que ces suggestions dépendent de votre objectif final et du modèle que vous prévoyez d'utiliser. Assurez-vous de comprendre les implications de chaque étape avant de l'appliquer à vos données. Bonne chance avec votre analyse ! 😊

In [ ]:
dfs2=dfs.copy()
# Prétraitement pour title_basics
dfs2['title_basics']['startYear'] = pd.to_numeric(dfs2['title_basics']['startYear'], errors='coerce')
dfs2['title_basics']['endYear'] = pd.to_numeric(dfs2['title_basics']['endYear'], errors='coerce')
dfs2['title_basics'] = dfs2['title_basics'].join(dfs2['title_basics']['genres'].str.get_dummies(','))

# Prétraitement pour title_crew
dfs2['title_crew'] = dfs2['title_crew'].join(dfs2['title_crew']['directors'].str.get_dummies(',').add_prefix('director_'))
dfs2['title_crew'] = dfs2['title_crew'].join(dfs2['title_crew']['writers'].str.get_dummies(',').add_prefix('writer_'))

# Prétraitement pour title_episode
dfs2['title_episode']['seasonNumber'] = pd.to_numeric(dfs2['title_episode']['seasonNumber'], errors='coerce')
dfs2['title_episode']['episodeNumber'] = pd.to_numeric(dfs2['title_episode']['episodeNumber'], errors='coerce')

# Prétraitement pour title_principals
dfs2['title_principals'] = dfs2['title_principals'].join(dfs2['title_principals']['characters'].str.get_dummies(',').add_prefix('character_'))

# Prétraitement pour title_ratings
dfs2['title_ratings']['averageRating'] = pd.to_numeric(dfs2['title_ratings']['averageRating'], errors='coerce')
dfs2['title_ratings']['numVotes'] = pd.to_numeric(dfs2['title_ratings']['numVotes'], errors='coerce')

# Fusion des tables
df = dfs2['title_basics'].merge(dfs2['title_crew'], on='tconst', how='left')
df = df.merge(dfs2['title_episode'], on='tconst', how='left')
df = df.merge(dfs2['title_principals'], on='tconst', how='left')
df = df.merge(dfs2['title_ratings'], on='tconst', how='left')

print(df.head())


In [ ]:
# KPI pour name_basics
print("Nombre total d'acteurs : ", dfs2['name_basics']['nconst'].nunique())
print("Nombre d'acteurs par profession : ", dfs2['name_basics']['primaryProfession'].value_counts())

# KPI pour title_basics
print("Nombre total de titres par type : ", dfs2['title_basics']['titleType'].value_counts())
print("Durée moyenne des titres : ", dfs2['title_basics']['runtimeMinutes'].mean())

# KPI pour title_ratings
print("Note moyenne des titres : ", dfs2['title_ratings']['averageRating'].mean())
print("Nombre moyen de votes par titre : ", dfs2['title_ratings']['numVotes'].mean())

# Suggestions de films
def suggest_movies(movie_title):
    # Sélectionnez le film choisi par l'utilisateur
    movie = dfs2['title_basics'][dfs2['title_basics']['primaryTitle'] == movie_title].iloc[0]

    # Identifiez les genres de ce film
    genres = movie['genres'].split(',')

    # Filtrez votre DataFrame pour ne garder que les films qui partagent au moins un genre avec le film choisi
    similar_movies = dfs2['title_basics'][dfs2['title_basics']['genres'].apply(lambda x: any(genre in x for genre in genres))]

    # Fusionnez avec title_ratings
    similar_movies = similar_movies.merge(dfs2['title_ratings'], on='tconst')

    # Classez ces films en fonction de leur note moyenne
    similar_movies = similar_movies.sort_values(by='averageRating', ascending=False)

    # Sélectionnez les 10 premiers films de cette liste
    top_10_movies = similar_movies.head(10)

    return top_10_movies

# Remplacez par le titre du film choisi par l'utilisateur
movie_title = "Carmencita"
print("Voici 10 suggestions de films basées sur le film ", movie_title, " :")
print(suggest_movies(movie_title))
